# アユムのファイル

In [1]:
!pip install ultralytics

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 6.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/772.7 kB ? eta -:--:--
   ---------------------------------------- 772.7/772.7 kB 6.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.1 MB ? eta -:--:--
   --- ------------------------------------ 3.4/40.1 MB 16.8 MB/s eta 0:00:03
   ------- -------------------------------- 7.6/40.1 MB 18.8 MB/s eta 0:00:02
   --------- ------------------------------ 9.7/40.1 MB 15.5 MB/s eta 0:00:02
   ------------- -------------------------- 13.4/40.1 MB 14.7 MB/s eta 0:00:02
   ---------------- ----------------------- 16.3/40.1 MB 14.0 MB/s eta 0:00:02
   ------------------ --------------------- 18.9/40.1 MB 13.9 MB/s eta 0:00:02
   --------------------- ------------------ 21.8/40.1 MB 13.8 MB/s eta 0:00:02
   ----------------------- ---------------- 23.3/40.1 MB 13.2 MB/s eta 0:00:02
   --


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-pose.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="coco-pose.yaml", epochs=100, imgsz=640)

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\agugu\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.216  Python-3.11.5 torch-2.7.1+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco-pose.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, ke

KeyboardInterrupt: 